In [3]:
import requests, time, csv, os
import configparser

# config.ini ファイルを読み込む
config = configparser.ConfigParser()
config.read('config.ini')

# 1inch API情報
API_KEY_1INCH = config.get('1inch', 'api_key')
TOKEN_ADDRESS = "0xd88ca08d8eec1e9e09562213ae83a7853ebb5d28"  # XWIN(BEP20)
USDT_ADDRESS = "0x55d398326f99059ff775485246999027b3197955"
AMOUNT = 5000

def get_quote(src, dst, amount, api_key):
    response = requests.get(
        "https://api.1inch.dev/swap/v6.0/56/quote",
        headers={"Authorization": f"Bearer {api_key}"},
        params={"src": src, "dst": dst, "amount": amount}
    )
    return response.json() if response.status_code == 200 else None

def calculate_rates(token_address, usdt_address, amount, api_key):
    quote = get_quote(token_address, usdt_address, amount, api_key)
    if not quote:
        print("Failed to get quote for token to USDT.")
        return None

    USDT_num = int(quote['dstAmount'])
    rate_token_to_usdt = round(USDT_num / amount, 4)
    print(f"Bid Rate: {rate_token_to_usdt:.4f}")
    time.sleep(1.1)

    quote = get_quote(usdt_address, token_address, USDT_num, api_key)
    if not quote:
        print("Failed to get quote for USDT to token.")
        return None

    XWIN_num = float(quote['dstAmount'])
    rate_usdt_to_token = round(USDT_num / XWIN_num, 4)
    print(f"Ask Rate: {rate_usdt_to_token:.4f}")
    mid_rate = round((rate_token_to_usdt + rate_usdt_to_token) / 2, 4)

    return mid_rate

mid_rate = calculate_rates(TOKEN_ADDRESS, USDT_ADDRESS, AMOUNT, API_KEY_1INCH)
if mid_rate:
    file_path = r'C:\Users\Administrator\xwin.csv'
    if os.path.isfile(file_path):
        print(f"The file '{file_path}' exists.")
    else:
        file_path = r'xwin.csv'
    with open(file_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Mid_Rate"])
        writer.writerow([mid_rate])
    print(f"Mid Rate: {mid_rate:.4f}")
else:
    print("No sufficient data to calculate.")


Bid Rate: 0.0116
Ask Rate: 0.0118
Mid Rate: 0.0117
